# Experiment Tracking and Model Management

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/ok-transformer/blob/master/docs/nb/mlops/02-mlflow)
[![Stars](https://img.shields.io/github/stars/particle1331/ok-transformer?style=social)](https://github.com/particle1331/ok-transformer)

```text
𝗔𝘁𝘁𝗿𝗶𝗯𝘂𝘁𝗶𝗼𝗻: Notes for Module 2 of the MLOps Zoomcamp (2022) by DataTalks.Club.
```

---

## Introduction

In this module, we will look **experiment tracking** and **model management**. A machine learning experiment is defined as a session or process of making machine learning models. Experiment tracking is the process of keeping track of all relevant information in an experiments. This includes source code, environment, data, models, hyperparameters, and so on which are important for reproducing the experiment as well as for making actual predictions. 

Manual tracking, e.g. with spreadsheets, is error prone, not standardized, has low visibility, and difficult for teams to collaborate over. As an alternative, we will use the experiment tracking platform [**MLflow**](https://mlflow.org/). As we have seen with prototyping a ride duration model, having the ability to reproduce results is important since we want to have the same results when deploying the model in different environments. Using experiment tracking and model management platforms allows us to have better chance at reproducing our results, as well as aid in model management and optimization.

## Getting started: MLflow UI

We can run the MLflow UI with an SQLite backend as follows:


```bash
$ mlflow ui --backend-store-uri sqlite:///mlflow.db

[2022-05-26 19:35:22 +0800] [92498] [INFO] Starting gunicorn 20.1.0
[2022-05-26 19:35:22 +0800] [92498] [INFO] Listening at: http://127.0.0.1:5000 (92498)
[2022-05-26 19:35:22 +0800] [92498] [INFO] Using worker: sync
[2022-05-26 19:35:22 +0800] [92499] [INFO] Booting worker with pid: 92499
```

For our experiment, we will use our code and data from [Module 1](https://particle1331.github.io/ok-transformer/nb/mlops/1-intro.html). So before doing any run, we either create an **experiment** or connect a run to it if the experiment already exists. This also sets the experiment tracking backend. The same one that is visualized in the UI above.

```{margin}
[`experiment_lr.py`](https://github.com/particle1331/ok-transformer/blob/mlops/docs/nb/mlops/02-mlflow/experiment_lr.py#L25-L27)
```

```python
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")
```

The next section of the code executes a **single run** of the experiment. Note the logging at the end of the script. This is quite involved, but here we want to show what types of data can be logged in MLflow. Everything that runs inside the following context is a single run:

```{margin}
[`experiment_lr.py`](https://github.com/particle1331/ok-transformer/blob/mlops/docs/nb/mlops/02-mlflow/experiment_lr.py#L30-L62)
```
```python
with mlflow.start_run():

    model = LinearRegression()
    model.fit(X_train, y_train)

    # MLflow logging
    start_time = time.time()
    y_pred_train = model.predict(X_train)
    y_pred_valid = model.predict(X_valid)
    inference_time = time.time() - start_time

    rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
    rmse_valid = mean_squared_error(y_valid, y_pred_valid, squared=False)

    fig = plot_duration_distribution(model, X_train, y_train, X_valid, y_valid)
    fig.savefig(artifacts / 'plot.svg')

    mlflow.set_tag('author', 'particle')
    mlflow.set_tag('model', 'baseline')
    
    mlflow.log_param('train_data_path', train_data_path)
    mlflow.log_param('valid_data_path', valid_data_path)
    
    mlflow.log_metric('rmse_train', rmse_train)
    mlflow.log_metric('rmse_valid', rmse_valid)
    mlflow.log_metric(
        'inference_time', 
        inference_time / (len(y_pred_train) + len(y_pred_valid))
    )
    
    mlflow.log_artifact(artifacts / 'plot.svg')
    mlflow.log_artifact(artifacts / 'preprocessor.pkl', artifact_path='preprocessing')
```

After running this script, we see in the UI as a single run under the `nyc-tax-experiment` experiment. MLflow is able to obtain the version from `git` and the user from the system, i.e. the user that is currently logged in. The other values are obtained from the logs. 

```{figure} ../../../img/single-run-mlflow.png
```

If we click on the run, we can see all details about it that were logged. Shown here are the date of the run, the user that executed it, total runtime, the source code used, as well as the git commit hash. Hence, it is best practice to always commit before running experiments. This ties your runs with a specific version of the code. Status `FINISHED` indicates that the script successfully ran. These are useful metadata.

```{figure} ../../../img/mlflow-single-run.png
---
---
Logged run of the baseline model. MLflow allows us to preview saved artifacts. Shown here is a plot of distribution of true and predicted target values.
```

Regarding the details of the trained model, we have parameters which here include only the path of the dataset for training and validation (only paths, no versioning). Most importantly, we can see the logged RMSEs `5.7` (train) and `7.759` (valid). The plot of the distributions of the true and predicted distributions which we logged as a training artifact is also conveniently displayed here. Finally, we log the preprocessor as **artifact** which will be needed for preprocessing test data during inference.

## Experiment tracking

In this section, we go deeper into experiment tracking with MLflow. We show how to iterate over different models and different parameters. This really just involves wrapping the run function in a loop. The nontrivial part is how to construct the sequence of parameters to loop over. For this we use [Hyperopt](https://hyperopt.github.io/hyperopt/) which implements the [TPE algorithm](https://proceedings.neurips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf). 

We also look at the **autologging** feature for supported frameworks. This automates logging of framework specific values such as intermediate values for models trained incrementally such as XGBoost or neural networks, or the estimator class for scikit-learn models. But more importantly, autologging generates the `MLModel` file which makes it easy to deploy models for downstream tasks. Really convenient feature.

### Using scikit-learn models

To evaluate different models, we will define a `run` function that takes in parameters that define and configure a run of the experiment. Then, we define a `main` function that controls the runs that will be executed. Here the `model_class` parameter controls which scikit-learn model is used to model the dataset. Note that this connects to the same tracking URI and same experiment.

```{margin}
[`experiment_sklearn.py`](https://github.com/particle1331/ok-transformer/blob/mlops/docs/nb/mlops/02-mlflow/experiment_sklearn.py)
```
```python
def setup():
    
    # Preprocessing
    ...

    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    mlflow.set_experiment("nyc-taxi-experiment")
    mlflow.sklearn.autolog()


def run(model_class):
    with mlflow.start_run():

        model = model_class()
        model.fit(X_train, y_train)

        # MLflow logging
        ...


def main():
    for model_class in [
        Ridge,
        RandomForestRegressor, 
        GradientBoostingRegressor,
    ]:
        run(model_class)


if __name__ == "__main__":
    setup()
    main()
```

`GradientBoostingRegressor` has the best validation score:

```{figure} ../../../img/sklearn-results.png
```

Note that autologging is available for scikit-learn models. Using this, parameters (even default ones) are automatically logged. Also, this generates the `MLModel` file along with the environments files and the serialized model. For scikit-learn models we can also see the `estimator_class` of the model. Finally, the input and output signature of the model is also stored. Autologging for supported frameworks can be activated by calling `mlflow.<framework>.autolog()`. This is a really convenient feature of MLflow.

```{figure} ../../../img/autolog-sk1.png
```

```{figure} ../../../img/autolog-sk2.png
---
---
Autologging of a Random Forest model in scikit-learn.
```

### Hyperparameter optimization (XGBoost)

From the previous runs, `GradientBoostingRegressor` has the best performance on this task. So we try out XGBoost next. Moreover, we find the best parameter of XGBoost using Hyperopt. Here the parameters are sampled using the TPE algorithm over the search space defined in the `search_space` dictionary. We look at functionalities MLflow provides to assist with hyperparameter optimization.

As before, we will define a `setup` function which sets up the required datasets and connections, define a run function (here named `objective`), and a `main` function to facilitate the runs. Finally, we define command line arguments, so we can easily control the details of the runs in the command line. 

```{margin}
[`experiment_xgboost.py`](https://github.com/particle1331/ok-transformer/blob/mlops/docs/nb/mlops/02-mlflow/experiment_xgboost.py)
```
```python
def setup():

    # Preprocessing
    ... 
    
    # Set experiment
    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    mlflow.set_experiment("nyc-taxi-experiment")
    mlflow.xgboost.autolog()


def objective(params):
    """Compute validation RMSE (one trial = one run)."""

    with mlflow.start_run():
        
        model = xgb.train(
            params=params,
            dtrain=xgb_train,
            num_boost_round=1000,
            evals=[(xgb_valid, 'validation')],
            early_stopping_rounds=50
        )

        # MLflow logging
        ...
    
    return {'loss': rmse_valid, 'status': STATUS_OK}


search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}


def main(num_runs):
    best_result = fmin(
        fn=partial(objective),
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_runs,
        trials=Trials()
    )


if __name__ == "__main__":
    
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--num_runs", default=1, type=int)
    args = parser.parse_args()
    
    # Experiment runs
    setup()
    main(num_runs=args.num_runs)
```

Let us look at the details of a single XGBoost run with autologging. Here all 14 parameters of XGBoost are logged (much more than what we have in our search space). Also, we have feature importances and the `MLModel` file along with the environments files. Moreover, we get metrics that are relevant for models trained incrementally, such as the best and stopped iterations (early stopped), as well as a plot of the intermediate values:

```{figure} ../../../img/xgboost-run.png
```

```{figure} ../../../img/xgboost-intermediate.png
---
---
Intermediate values plot for an autologged XGBoost model.
```

We executed 30 runs of the XGBoost model (+1 with autologging). This can be analyzed using the compare button after selecting the results with the search query `tags.model = 'xgboost'`. Below we analyze the runs with the parallel coordinate plot. First, we have the specify the parameters and the metric. Then, we can use filters to filter out ranges that result in low objective values. 

```{figure} ../../../img/xgboost-parallel.png
```

In the other tabs, we also have scatter and contour interactive [plotly](https://plotly.com/) plots:

```{figure} ../../../img/xgboost-scatter.png
```

```{figure} ../../../img/xgboost-contour.png
---
---
Plots for analyzing the hyperparameter search space of an XGBoost model on this dataset.
```

## Model management

In this section, we take a deeper look at **model management**. In addition to experiment tracking, part of model management is to do model versioning and model deployment. Model management using file systems, typically involving only a file name and date modified, is error prone. There is no versioning and no [model lineage](https://aws.amazon.com/blogs/machine-learning/model-and-data-lineage-in-machine-learning-experimentation/). Model lineage refers to all associations between a model and all components involved in its creation.
Having no model lineage makes it difficult to track results and progress, not to mention reproducing them. 

For each run a `run_id` key is assigned which maps to information such as metrics, source version, and other metadata in the database. If we make sure to commit the code before running model training, then we can trace the source for each run. Each run also has an `artifacts` directory which saves in disk all logged artifacts. 

Autologging also generates a [`MLModel` file](https://www.mlflow.org/docs/latest/models.html) which provides a standard format for packaging models that can be used in a variety of downstream tools (e.g. serving through a REST API or batch inference on Apache Spark). This includes package dependencies and Python version used for training, as well as the input and output signature of the model. 

This should allow you to reproduce results of experiment runs with relative ease. Indeed, below we perform inference using the stored preprocessing pipeline and load the model from the `MLModel` file. Note that the full path can be obtained by clicking on the copy button. 

```{figure} ../../../img/autologging-1.png
---
---
In addition to run metadata, the `MLModel` file obtained with autologging preserves model lineage.
```

### Model file loading

In the following code, we try to perform inference on test data using the logged model and artifacts. Note that new data has no labels when processed for inference. Let us try to reproduce the valid RMSE of `6.656`. Recall that the model was validated only for rides between 1 and 60 minutes in duration.

In [30]:
import pandas as pd
import mlflow
import joblib
from utils import runs, data_path, compute_targets
from sklearn.metrics import mean_squared_error


# Configure paths to run
run_path = runs / '1' / 'f46f9fedb22c4411b6e265f8e65edbe3'
model_path = run_path / 'artifacts' / 'model'
model_artifacts_path = run_path / 'artifacts' / 'preprocessing'
valid_data_path = data_path / 'green_tripdata_2021-02.parquet'

# Preprocessing test data
feature_pipe = joblib.load(model_artifacts_path / 'preprocessor.pkl')
X = pd.read_parquet(valid_data_path)
y = compute_targets(X)

X = X[(y >= 1) & (y <= 60)]
y = y[(y >= 1) & (y <= 60)]
X = feature_pipe.transform(X)

# Inference using MLModel file
loaded_model = mlflow.pyfunc.load_model(model_path)

# Reproducing metric: expected 6.656
print(mean_squared_error(y, loaded_model.predict(X), squared=False))

6.655558028101635


### Model registry

Consider the scenario where a data scientist member of our team chooses a new model for production. As deployment engineers, we naturally have the following questions in mind: What has changed in this new model? Is there any preprocessing needed? What are the dependencies? Without experiment tracking, this requires a lot of back and fort communication with the data scientist. If there is an incident in production and we had to rollback the model version, we have to go back to the email thread to determine what changed in this new version. Moreover, it might not be possible to get back the previous model (information of how it was trained has been lost). 

Recall that having a experiment tracking database and standard model files, solves most of the problems above. Having a **model registry** takes care of the last details of versioning and staging. For example, if we want to rollback models, we can only look at the archived models. Note that the model registry does not perform actual deployment of models, it only serves as standardization the process of moving models to production. Further integration with a CI/CD pipeline is needed if we want to trigger deployment steps along with the change in registry labels.

```{margin}
[Source: `Databricks`](https://databricks.com/blog/2020/04/15/databricks-extends-mlflow-model-registry-with-enterprise-features.html)
```
```{figure} ../../../img/model-registry-diagram.png
---
width: 35em
---
Having a model registry allows separation of concern. The data scientist only decides which models are ready for production while the deployment engineer takes care of moving models from staging to production, from production to archived.
```

To select the models, first we filter with `metrics.rmse_valid < 6.4`. Then, we can look at the tradeoff between valid RMSE with inference time. The scatter plot is the best tool for this. 
The tradeoffs should be weighed against production requirements. Next step is to analyze the subsets of the data where these top models are wrong, perhaps create an ensemble of them, and so on.

```{figure} ../../../img/xgboost-select.png
```

```{figure} ../../../img/xgboost-tradeoff.png
```

From this plot, we look at the 5 points within `20e-6` inference time and `6.32` valid RMSE. For the sake of demonstration, these will make up different versions of our model. The rightmost model in the scatter plot is version 1 of ride duration prediction model. To register this, we simply click the `Register Model` button in the UI. For our model name, we choose `nyc-ride-duration-model` and we stage this is production. The model registry can be viewed in the `Models` tab of the MLflow UI. We can register any other model to update the existing model.

```{figure} ../../../img/register-model.png
```

```{figure} ../../../img/staging.png
---
---
Registering a model and staging a new version of the model trained on the same task.
```


```{figure} ../../../img/production.png
---
---
Transitioning the staged model to production and the current model to archived.
```


## API Workflows

In this section, we look at how interact with the MLflow through an API. The idea is that everything that we can do in the UI by clicking buttons, we should be able to do here in code. Also, the fields that are available when using the UI correspond to the function arguments of the corresponding API endpoint.

### Experiment tracking

The `MlflowClient` connects to the experiment tracking server. This provides a [CRUD interface](https://en.wikipedia.org/wiki/Create,_read,_update_and_delete) for managing experiments and runs. To demonstrate, we create an experiment using the client and then fetch this experiment by name.

In [37]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


def print_experiment(experiment):
    print(f"(Experiment)")
    print(f"    experiment_id={experiment.experiment_id}")
    print(f"    name='{experiment.name}'")
    print(f"    artifact_location='{experiment.artifact_location}'")
    print()


for experiment in client.list_experiments():
    print_experiment(experiment)

(Experiment)
    experiment_id=0
    name='Default'
    artifact_location='./mlruns/0'

(Experiment)
    experiment_id=1
    name='nyc-taxi-experiment'
    artifact_location='./mlruns/1'



In [39]:
client.create_experiment(name='test-create-experiment')
print_experiment(client.get_experiment_by_name('test-create-experiment'))

(Experiment)
    experiment_id=7
    name='test-create-experiment'
    artifact_location='./mlruns/7'



In the previous section, we filtered out the five best runs using the UI. This can be done with the client using the `search_runs` method. Note that MLflow stores even deleted experiments. So we specify `ViewType` to `ACTIVE_ONLY` to include only runs that are active (i.e. not deleted) in the search results.

In [42]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=1,
    filter_string='metrics.rmse_valid < 6.32 and metrics.inference_time < 20e-6',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse_valid ASC"]
)

for run in runs:
    print(f"run_id: {run.info.run_id}   rmse_valid: {run.data.metrics['rmse_valid']:.3f}   inference_time: {run.data.metrics['inference_time']:.4e}")

run_id: 1e5a211d4dd84158ade0f3131a6b467a   rmse_valid: 6.286   inference_time: 1.6639e-05
run_id: a5c39ab3d68d4de791529d083edab919   rmse_valid: 6.293   inference_time: 1.1944e-05
run_id: f167aef23eef4305b798771a76980bd3   rmse_valid: 6.294   inference_time: 5.9997e-06
run_id: 167836ec92094ac2b395387619e0a3b8   rmse_valid: 6.295   inference_time: 1.4946e-05
run_id: bc2c191810a646f287c428126b75f6eb   rmse_valid: 6.307   inference_time: 1.3798e-05


### Model registry

The following code puts a model into the registry. If the model name already exists, then we get a new version. Otherwise it creates a new model in the model registry. Below we create a new version of our model with almost the same error rate and half the inference time of the current model in production. Observe that MLflow likes to work with the latest version for each stage.

In [196]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
run_id = 'f167aef23eef4305b798771a76980bd3'

mlflow.register_model(
    model_uri=f"runs:/{run_id}/model", 
    name='nyc-ride-duration-model'
);

Registered model 'nyc-ride-duration-model' already exists. Creating a new version of this model...
2022/05/31 20:11:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-ride-duration-model, version 3
Created version '3' of model 'nyc-ride-duration-model'.


This version can be transitioned to staging as follows:

In [198]:
import datetime

def transition_stage(client, model_name, model_version, new_stage, archive_existing=False):
    """Transition version stage. Update description."""

    old_description = client.get_model_version(name=model_name, version=model_version).description or ''

    client.transition_model_version_stage(
        name=model_name,
        version=model_version,
        stage=new_stage,
        archive_existing_versions=archive_existing
    )

    client.update_model_version(
        name=model_name,
        version=model_version,
        description=f"[{datetime.datetime.now()}] The model version {model_version} was transitioned to {new_stage}.\n{old_description}"
    )


transition_stage(client, 'nyc-ride-duration-model', 3, "Staging")

```{figure} ../../../img/update-version.png
```


The following lists the latest versions for each stage:

In [199]:
latest_versions = client.get_latest_versions(name='nyc-ride-duration-model')
for version in latest_versions:
    print(f"Version: {version.version}   Run ID: {version.run_id}   Stage: {version.current_stage}" )

Version: 1   Run ID: bc2c191810a646f287c428126b75f6eb   Stage: Archived
Version: 2   Run ID: 167836ec92094ac2b395387619e0a3b8   Stage: Production
Version: 3   Run ID: f167aef23eef4305b798771a76980bd3   Stage: Staging


The same transition function can be used to promote Version 3 to production and archive Version 2 which is currently in production. Note that we can automatically archive Version 2. But we use the transition function so that the description is automatically updated.

In [200]:
transition_stage(client, 'nyc-ride-duration-model', 3, "Production")
transition_stage(client, 'nyc-ride-duration-model', 2, "Archived")

```{figure} ../../../img/final-versions.png
```

```{figure} ../../../img/version3-prod.png
```

```{figure} ../../../img/version2-archived.png
```

### Inference with staged models

We can make inference using models in specific stages. Here we use the production model to perform inference. This can be useful if we want to compare a production model to new models in staging. Or if we want to monitor the performance of the production model on new data.

In [63]:
from utils import runs, filter_target_outliers
from sklearn.metrics import mean_squared_error

# Prepare datasets for testing
valid_data_path = data_path / 'green_tripdata_2021-02.parquet'

X = pd.read_parquet(valid_data_path)
y = compute_targets(X)
X, y = filter_target_outliers(X, y)

# Fix paths
model_name = 'nyc-ride-duration-model'
stage = "Production"
run_id = client.get_latest_versions(name=model_name, stages=[stage])[0].run_id
artifacts_path = client.download_artifacts(run_id=run_id, path='preprocessing')

# Fetching the preprocessor and staged model (loads latest)
preprocessor = joblib.load(f"{artifacts_path}/preprocessor.pkl")
prod_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")

%time
mean_squared_error(y, prod_model.predict(preprocessor.transform(X)), squared=False)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 1.91 µs


6.293886787130318

<br>

**Remark.** MLflow seems to like working only with latest versions in the registry. For example, the line `mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")` above silently loads the latest model at the given stage. Typically, you would like to have only one model in the production stage, but it could be useful to have multiple models in staging. A workaround would be to retrieve the model by using the model version, i.e. use `f"models:/{model_name}/{model_version}"` as `model_uri`. Or fetch all versions of a registered model using:

```python
model_versions = client.search_model_versions(f"name='{model_name}'")
```

## Remote tracking server and artifact store on AWS

For storing runs and artifacts, MLflow uses two components for storage: **backend store** and **artifact store**. The backend store persists MLflow entities such as runs, parameters, metrics, tags, notes, metadata. The artifact store persists artifacts such as files, models, images, in-memory objects. In our previous local setup, MLflow runs are recorded in a local SQLite database, and artifacts persisted as local files in our file system. But the backend can be set to any SQLAlchemy compatible database, and MLflow artifacts can be persisted a variety of remote file storage solutions.

In this section, we configure MLflow with a remote backend and artifacts store, and a remote tracking server in AWS. This setup is useful for a multiple data scientists in a team working on multiple ML models. Here we will configure: 1) an S3 bucket as artifacts store, 2) a PostgreSQL database in RDS as backend store, and 3) an EC2 instance for running the remote tracking server. The experiments can then be explored by accessing the remote server.

```{margin}
[`Scenario 4`](https://www.mlflow.org/docs/latest/tracking.html#scenario-4-mlflow-with-remote-tracking-server-backend-and-artifact-stores)
```
```{figure} ../../../img/mlflow-scenario-4.png
---
---
```

### EC2 Instance

**Launch instance.** Start by logging to your IAM account. Here we launch a `t2.micro` EC2 instance with name `mlflow-tracking-server` and with `Amazon Linux 2 AMI (HVM))` 64-bit (x86) OS. These options are both free tier eligible. Create a new [key pair](https://particle1331.github.io/ok-transformer/nb/mlops/1-intro.html#renting-an-ec2-instance) so you can connect using SSH. Keep the default values for the other settings. Review the summary and launch the instance.

**Inbound rules.** Go to security groups and edit inbound rules. Our instance should accept incoming SSH (port 22) and HTTP connections (port 5000). Specify CIDR blocks to specify the range of IP addresses that has access to the tracking server, e.g. choose 'My IP' as source so only computers on your local network can access the server.

<br>

```{figure} ../../../img/inbound-rules.png
---
name: ec2-inbound
---
Choosing `0.0.0.0/0` allows all incoming HTTP access.
```

### S3 Bucket

**Create bucket.** Next we create the artifacts storage. Go to S3 and click on 'Create bucket'. Fill in the bucket name with something that is unique in the all regions. In our case, we found `mlflow-artifacts-store-2` to be unique. Leave all the other configurations with their default values. That's it!

### PostgreSQL DB

**Create database.** Go to the RDS Console and click on 'Create database'. Make sure to select PostgreSQL engine type and the Free tier template. Select an identifier for your DB instance, e.g. `mlflow-backend-db` shown here. Set the master username as `mlflow`. Tick the option 'Auto generate a password' so Amazon RDS generate a password automatically. Finally, on the section 'Additional configuration' specify an initial database name, e.g. `mlflow_backend_db`, so RDS automatically creates an initial database for you. The generated password will be shown (only once) after the database has been created. You can use the default values for all the other configurations.


<br>


```{figure} ../../../img/rds-template.png
---
width: 40em
---
Choosing an engine and template.
```

```{figure} ../../../img/rds-identifier.png
---
width: 40em
---

```

```{figure} ../../../img/rds-name.png
---
width: 40em
---
Choosing identifier, username, password, and initial database name. 
```

**Credentials.** Take note of the following information: master username (`DB_USERNAME`), password (`DB_PASSWORD`), initial database name (`DB_NAME`), and endpoint (`DB_ENDPOINT`). On the RDS dashboard, click on the database and look at 'Connectivity & security'. The endpoint for the database can be seen there. 

**Inbound rules.** Select the VPC security group of the DB under the same tab. Click the security group ID, and edit inbound rules by adding a new rule that allows PostgreSQL connections on the port 5432 from the security group of the EC2 instance for the tracking server. Note that the security group of the EC2 instance is `sg-049fa46c4cd030db2 - launch-wizard-2` as can be seen on the top of {numref}`ec2-inbound`. This way, the tracking server will be able to connect to the PostgreSQL database. 

<br>

```{figure} ../../../img/postgres.png
---
width: 40em
---

Allow postgres connections from the tracking server to the backend database.

```


### Server config and launch

Here we connect to the `mlflow-tracking-server` EC2 instance from our local terminal. Change to the local `.ssh` directory then:

```bash
chmod 400 mlflow-tacking.pem
ssh -i "mlflow-tracking.pem" ec2-user@ec2-34-209-62-152.us-west-2.compute.amazonaws.com
```

Run the following commands to install the dependencies, configure the environment on the EC2 instance:

```
sudo yum update
pip3 install mlflow boto3 psycopg2-binary
aws configure   # Input your IAM credentials here
aws s3 ls       # Test if instance can access S3 bucket
```

Finally, launch the server by replacing the following with the appropriate database credentials. You can replace the variables below manually, or you can execute the following assignments for each variable, so that the values are automatically filled.

```bash
export DB_USER=mlflow
export DB_PASSWORD=ZbTddA0Zc8LxYcdLFUQr
export DB_ENDPOINT=mlflow-backend-database.csegt7oxppl.us-west-2.rds.amazonaws.com
export DB_NAME=mlflow_backend_database
export S3_BUCKET_NAME=mlflow-artifact-store-2

mlflow server -h 0.0.0.0 -p 5000 \
    --backend-store-uri=postgresql://${DB_USER}:${DB_PASSWORD}@${DB_ENDPOINT}:5432/${DB_NAME} \
    --default-artifact-root=s3://${S3_BUCKET_NAME}
```

### Trying it out

Since we are allowed by the inbound rules to send information via HTTP to the tracking server, we should be able to send requests to it. Note that everything that follows is done in our local Jupyter notebook. After setting the tracking URI we should be able to manage experiments and the model registry through the client as before. Note that you may have to do `aws configure` for credentials in an external machine. 

In [83]:
import mlflow
from mlflow.tracking import MlflowClient


TRACKING_SERVER_HOST = "ec2-34-209-62-152.us-west-2.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

client = MlflowClient(tracking_uri=f"http://{TRACKING_SERVER_HOST}:5000")
print(client.list_experiments())
print(client.list_registered_models())

[]
[]


Running an example experiment.

In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


mlflow.set_experiment("iris")
X, y = load_iris(return_X_y=True)

for C in [10, 1, 0.1, 0.01, 0.001]:
    with mlflow.start_run(nested=True):

        params = {"C": C, "random_state": 42}
        
        lr = LogisticRegression(**params).fit(X, y)
        y_pred = lr.predict(X)

        mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
        mlflow.log_params(params)
        mlflow.sklearn.log_model(lr, artifact_path="models")

2022/06/04 03:44:56 INFO mlflow.tracking.fluent: Experiment with name 'iris' does not exist. Creating a new experiment.


<br>

```{figure} ../../../img/aws-runs.png
---
width: 40em
---

Runs are recorded in the tracking UI.

```


```{figure} ../../../img/s3.png
---
width: 40em
---

We can see the runs artifacts stored in the S3 bucket.

```


## Appendix: Utility code

For the experiment run scripts, we used the following utility script to process the datasets used for training. Here we define a preprocessing pipeline `feature_pipe` which allows saving a single preprocessor file as artifact. This also keeps the code clean (i.e. the `transform` method is called only on the raw datasets).

```python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from toolz import compose
from pathlib import Path
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')


# Config variables
root = Path(__file__).parent.resolve()
artifacts = root / 'artifacts'
artifacts.mkdir(exist_ok=True)
runs = root / 'mlruns'
data_path = root / 'data'


class PrepareFeatures(BaseEstimator, TransformerMixin):
    """Prepare features for dict vectorizer."""

    def __init__(self, categorical, numerical):
        self.categorical = categorical
        self.numerical = numerical
        self.features = categorical + numerical

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['PU_DO'] = X['PULocationID'].astype(str) + '_' + X['DOLocationID'].astype(str)
        X[self.categorical] = X[self.categorical].astype(str)
        X = X[self.features].to_dict(orient='records')
        
        return X


def compute_targets(data):
    """Derive target from pickup and dropoff datetimes."""

    # Create target column and filter outliers
    data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data['duration'] = data.duration.dt.total_seconds() / 60
    
    targets = data.duration.values
    return targets


def filter_target_outliers(data, targets, y_min=1, y_max=60):
    """Filter data with targets outside of range."""

    X = data[(data.duration >= y_min) & (data.duration <= y_max)]
    y = X.duration.values

    return X, y


def plot_duration_distribution(model, X_train, y_train, X_valid, y_valid):
    """Plot true and prediction distribution."""
    
    fig, ax = plt.subplots(1, 2, figsize=(8, 3))

    sns.histplot(model.predict(X_train), ax=ax[0], label='pred', color='C0', stat='density', kde=True)
    sns.histplot(y_train, ax=ax[0], label='true', color='C1', stat='density', kde=True)
    ax[0].set_title("Train")
    ax[0].legend()

    sns.histplot(model.predict(X_valid), ax=ax[1], label='pred', color='C0', stat='density', kde=True)
    sns.histplot(y_valid, ax=ax[1], label='true', color='C1', stat='density', kde=True)
    ax[1].set_title("Valid")
    ax[1].legend()

    fig.tight_layout()
    return fig


def preprocess_datasets(train_data_path, valid_data_path):
    """Preprocess datasets for model training and validation. Save pipeline."""
 
    X_train = pd.read_parquet(train_data_path)
    X_valid = pd.read_parquet(valid_data_path)
    
    # Compute labels
    y_train = compute_targets(X_train)
    y_valid = compute_targets(X_valid)

    # Filter train and valid (!) data. (i.e. only validate on t=(1, 60) range.)
    X_train, y_train = filter_target_outliers(X_train, y_train)
    X_valid, y_valid = filter_target_outliers(X_valid, y_valid)
    
    # Feature selection and engineering
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    feature_pipe = make_pipeline(
        PrepareFeatures(categorical, numerical),
        DictVectorizer(),
    )

    # Fit only on train set
    feature_pipe.fit(X_train, y_train)
    X_train = feature_pipe.transform(X_train)
    X_valid = feature_pipe.transform(X_valid)

    # Save preprocessor
    joblib.dump(feature_pipe, artifacts / 'preprocessor.pkl')

    return X_train, y_train, X_valid, y_valid
```